In [1]:
from ics import Calendar
import csv
import pandas as pd
import re

In [2]:
def preprocess(input_path, output_path):
    """
    Adjust the formatting of LOCATION lines so that the next two lines are indented.
    """
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    processed_lines = []
    i = 0
    while i < len(lines):
        line = lines[i]
        processed_lines.append(line)

        # Check if this line is a LOCATION line
        if line.startswith("LOCATION:"):
            # Indent the next two lines if they exist
            if i + 1 < len(lines) and not lines[i + 1].startswith(" "):
                processed_lines.append(" " + lines[i + 1].strip() + "\n")
                i += 1
            if i + 1 < len(lines) and not lines[i + 1].startswith(" "):
                processed_lines.append(" " + lines[i + 1].strip() + "\n")
                i += 1
        i += 1

    # Write the processed lines to a new file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.writelines(processed_lines)

In [3]:
def parse_ics_to_csv(ics_file, csv_file):
    """Parse ICS file and export events to CSV."""
    with open(ics_file, 'r', encoding='utf-8') as f:
        calendar = Calendar(f.read())

    # Open CSV file for writing
    with open(csv_file, mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Event Name', 'Start Date', 'End Date', 'Location', 'Description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for event in calendar.events:
            writer.writerow({
                'Event Name': event.name,
                'Start Date': event.begin.format('YYYY-MM-DD HH:mm:ss') if event.begin else '',
                'End Date': event.end.format('YYYY-MM-DD HH:mm:ss') if event.end else '',
                'Location': event.location or '',
                'Description': event.description or ''
            })

In [4]:
input_ics = "../icals/kampe5db2b479-c637-46cf-b23b-6fb2736f5352.ics"
fixed_ics = "fixed/fixed_calendar.ics"
output_csv = "csvs/calendar_events.csv"

In [85]:
df_cal = pd.DataFrame(columns=['Årgang', 'Dag', 'Dato', 'Tidspunkt', 'Række', 'Kampnr', 'Hjemme', 'Ude', 'Region'])
df_cal

,Årgang,Dag,Dato,Tidspunkt,Række,Kampnr,Hjemme,Ude,Region


In [ ]:
import locale
from datetime import datetime
def parse_entry(content):
    try:
        # Regex patterns to extract fields
        kampnr_pattern = r"Kampnr (\d+)"
        date_time_pattern = r"(\d{2}-\d{2}-\d{4}) kl\. (\d{2}:\d{2})"

        # Extract data using regex
        kampnr_match = re.search(kampnr_pattern, content)
        date_time_match = re.search(date_time_pattern, content)

        # Assign default values if matches are missing
        kampnr = kampnr_match.group(1) if kampnr_match else "Unknown"
        dato, tidspunkt = (
            (date_time_match.group(1), date_time_match.group(2))
            if date_time_match
            else ("Unknown", "Unknown")
        )

        # Extract league name (Række) and Årgang
        række = content.split("\n")[0].strip()
        årgang = content.split("\n")[0].split(" ")[0]  # Hardcoded based on the description

        # Find the day of the week
        locale.setlocale(locale.LC_TIME, "da_DK.UTF-8")
        date_string = "10-11-2024"  # Format: DD-MM-YYYY

        # Parse the date
        date_object = datetime.strptime(date_string, "%d-%m-%Y")

        # Get the day of the week
        dag = date_object.strftime("%A").capitalize()  # Full day name (e.g., Saturday)

        # Home and away teams
        hjem = content.split("\n")[3].split(" - ")[0]
        ude = content.split("\n")[3].split(" - ")[1]

        # Region
        postnr = content.split("\n")[7].split(" ")[0]
        postnr = int(postnr)
        region = (
            "Øst" if postnr < 5000 else "Vest"
        )

        return [årgang, dag, dato, tidspunkt, række, kampnr, hjem, ude, region]
    except Exception as e:
        print(f"Error parsing entry: {content}\n{e}")
        return None

In [103]:
file_path = 'csvs/calendar_events2.csv'
with open(file_path, 'r', encoding='utf-8') as f:
    raw_data = f.read()

# Split the data into entries by matching content between double quotes
entries = re.findall(r'"(.*?)"', raw_data, re.DOTALL)

In [104]:
for entry in entries:
    parsed_row = parse_entry(entry)
    if parsed_row:
        df_cal.loc[len(df_cal)] = parsed_row

In [108]:
df_cal = df_cal.applymap(lambda x: x.replace("��", "ø") if isinstance(x, str) else x)


In [109]:
df_cal.loc[0:15]

,Årgang,Dag,Dato,Tidspunkt,Række,Kampnr,Hjemme,Ude,Region
0,U14,Søndag,09-11-2024,15:00,U14 Drenge Liga Øst 1 (11) helår,913470,Farum BK,OB,Øst
1,U14,Søndag,29-09-2024,15:00,U14 Drenge Liga Øst 1 (11) helår,913461,Farum BK,KB,Øst
2,U14,Søndag,28-09-2024,13:00,U14 Drenge Liga Øst 1 (11) helår,913462,Lyngby Boldklub 1921,Brøndbyernes IF,Øst
3,U14,Søndag,28-09-2024,14:00,U14 Drenge Liga Øst 1 (11) helår,913463,B.93,HB Køge,Øst
4,U14,Søndag,17-04-2025,12:00,U14 Drenge Liga Øst 1 (11) helår,913464,Brøndbyernes IF,KB,Øst
5,U14,Søndag,06-10-2024,13:30,U14 Drenge Liga Øst 1 (11) helår,913465,HB Køge,OB,Øst
6,U14,Søndag,05-10-2024,13:30,U14 Drenge Liga Øst 1 (11) helår,913466,Lyngby Boldklub 1921,Farum BK,Øst
7,U14,Søndag,06-10-2024,14:00,U14 Drenge Liga Øst 1 (11) helår,913467,HIK,B.93,Øst
8,U14,Søndag,08-03-2025,10:00,U14 Drenge Liga Øst 1 (11) helår,913468,KB,HIK,Øst
9,U14,Søndag,09-11-2024,11:00,U14 Drenge Liga Øst 1 (11) helår,913469,HB Køge,Lyngby Boldklub 1921,Øst


In [111]:
df_cal.to_excel('excels/calendar_events2.xlsx', index=False, engine='openpyxl')

In [122]:
df_cal['Dato'] = pd.to_datetime(df_cal['Dato'], format='%d-%m-%Y') 
df_cal.sort_values(by=['Dato', 'Tidspunkt'], ascending=[True, True], inplace=True)
df_cal.head(15)



,Årgang,Dag,Dato,Tidspunkt,Række,Kampnr,Hjemme,Ude,Region
57,U14,Søndag,2024-08-17,14:00,U14 Drenge Liga Øst 1 (11) helår,913417,B.93,HIK,Øst
141,U14,Søndag,2024-08-17,14:00,U14 Drenge Liga Øst 1 (11) helår,913417,B.93,HIK,Øst
55,U14,Søndag,2024-08-18,14:00,U14 Drenge Liga Øst 1 (11) helår,913415,HB Køge,OB,Øst
139,U14,Søndag,2024-08-18,14:00,U14 Drenge Liga Øst 1 (11) helår,913415,HB Køge,OB,Øst
56,U14,Søndag,2024-08-18,15:00,U14 Drenge Liga Øst 1 (11) helår,913416,Farum BK,Lyngby Boldklub 1921,Øst
140,U14,Søndag,2024-08-18,15:00,U14 Drenge Liga Øst 1 (11) helår,913416,Farum BK,Lyngby Boldklub 1921,Øst
35,U14,Søndag,2024-08-21,18:30,U14 Drenge Liga Øst 1 (11) helår,913420,Lyngby Boldklub 1921,HB Køge,Øst
119,U14,Søndag,2024-08-21,18:30,U14 Drenge Liga Øst 1 (11) helår,913420,Lyngby Boldklub 1921,HB Køge,Øst
19,U14,Søndag,2024-08-24,13:30,U14 Drenge Liga Øst 1 (11) helår,913433,Lyngby Boldklub 1921,HIK,Øst
103,U14,Søndag,2024-08-24,13:30,U14 Drenge Liga Øst 1 (11) helår,913433,Lyngby Boldklub 1921,HIK,Øst
